In [1]:
import requests
from datasets import load_dataset

ds = load_dataset("hihihohohehe/vifactcheck-normalized")

In [2]:
def generate_prompt(claim_text):
    return f"""Phân loại đoạn văn tiếng Việt dưới đây vào MỘT chủ đề duy nhất.

Chủ đề:
1 = CHÍNH TRỊ - PHÁP LUẬT - QUÂN SỰ
2 = KINH TẾ - BẤT ĐỘNG SẢN
3 = GIÁO DỤC - KHOA HỌC CÔNG NGHỆ
4 = SỨC KHỎE - XÃ HỘI ĐỜI SỐNG
5 = VĂN HÓA - GIẢI TRÍ - GIỚI TRẺ
6 = THỂ THAO
7 = DU LỊCH
8 = THẾ GIỚI - THỜI SỰ

Quy tắc:
- Chỉ trả về MỘT số từ 1 đến 8.
- Không giải thích, không ký tự khác.

Văn bản:
{claim_text}

Kết quả:"""


In [3]:
VALID_LABELS = {
    1: "CHÍNH TRỊ - PHÁP LUẬT - QUÂN SỰ",
    2: "KINH TẾ - BẤT ĐỘNG SẢN",
    3: "GIÁO DỤC - KHOA HỌC CÔNG NGHỆ",
    4: "SỨC KHỎE - XÃ HỘI ĐỜI SỐNG",
    5: "VĂN HÓA - GIẢI TRÍ - GIỚI TRẺ",
    6: "THỂ THAO",
    7: "DU LỊCH",
    8: "THẾ GIỚI - THỜI SỰ"}

In [8]:
URL = "http://localhost:11434/api/generate"

def ask_qwen(prompt):
    r = requests.post(
        URL,
        json={
            "model": "qwen2.5:1.5b",
            "prompt": prompt,
            "stream": False,
            "options": {
                "temperature": 0.0,
                "num_ctx": 2048,
                "num_predict": 10
            }
        },
        timeout=120
    )
    r.raise_for_status()
    label = r.json()["response"].strip()

    return label

In [17]:
import time

valid_answer = 0
times = []

for i in range(len(ds['train'])):
    text = ds['train'][i]['Statement']
    right_topic_id = ds['train'][i]['Topic ID']
    
    start = time.perf_counter()
    topic = ask_qwen(generate_prompt(text))
    end = time.perf_counter()

    times.append(end - start)

    if int(topic) == int(right_topic_id):
        valid_answer += 1
    
    print(f"Sample {i+1}/{len(ds['train'])} - Correct topic: {right_topic_id}, Predicted topic: {topic}, Valid answers: {valid_answer}, Accuracy: {valid_answer/(i+1):.4f}, Time taken: {end - start:.2f}s")


Sample 1/5062 - Correct topic: 1, Predicted topic: 1, Valid answers: 1, Accuracy: 1.0000, Time taken: 2.39s
Sample 2/5062 - Correct topic: 1, Predicted topic: 1, Valid answers: 2, Accuracy: 1.0000, Time taken: 4.37s
Sample 3/5062 - Correct topic: 4, Predicted topic: 1, Valid answers: 2, Accuracy: 0.6667, Time taken: 6.52s
Sample 4/5062 - Correct topic: 3, Predicted topic: 2, Valid answers: 2, Accuracy: 0.5000, Time taken: 6.69s
Sample 5/5062 - Correct topic: 3, Predicted topic: 1, Valid answers: 2, Accuracy: 0.4000, Time taken: 5.26s
Sample 6/5062 - Correct topic: 3, Predicted topic: 1, Valid answers: 2, Accuracy: 0.3333, Time taken: 4.62s
Sample 7/5062 - Correct topic: 6, Predicted topic: 2, Valid answers: 2, Accuracy: 0.2857, Time taken: 5.18s
Sample 8/5062 - Correct topic: 5, Predicted topic: 1, Valid answers: 2, Accuracy: 0.2500, Time taken: 4.22s
Sample 9/5062 - Correct topic: 5, Predicted topic: 4, Valid answers: 2, Accuracy: 0.2222, Time taken: 3.73s
Sample 10/5062 - Correct top

KeyboardInterrupt: 

In [18]:
ds

DatasetDict({
    train: Dataset({
        features: ['Unnamed: 0', 'index', 'Statement', 'Context', 'annotation_id', 'Topic', 'Author', 'Url', 'labels', 'Evidence', 'New Topic 1', 'New Topic 2', 'Topic ID'],
        num_rows: 5062
    })
    dev: Dataset({
        features: ['Unnamed: 0', 'index', 'Statement', 'Context', 'annotation_id', 'Topic', 'Author', 'Url', 'labels', 'Evidence', 'New Topic 1', 'New Topic 2', 'Topic ID'],
        num_rows: 723
    })
    test: Dataset({
        features: ['Unnamed: 0', 'index', 'Statement', 'Context', 'annotation_id', 'Topic', 'Author', 'Url', 'labels', 'Evidence', 'New Topic 1', 'New Topic 2', 'Topic ID'],
        num_rows: 1447
    })
})